In [4]:
from datasets import load_dataset, Dataset, Audio
import glob
import torch
import torchaudio

In [5]:
sources = glob.glob("MYRiAD_V2/audio/SRC/*.wav")
cocktail_party = [f"CP{i}.wav" for i in range(6)];
source_list = [suffix[-6:] for suffix in sources] + cocktail_party 
inner = [f"CMA10_{a}_" for a in [0,90,180,-90]]
outer = [f"CMA20_{a}_" for a in [0,45,90,135,180,-135,-90,-45]]
channel_list = inner + outer
ref_channel = channel_list[0]

In [6]:
audio = []
for source in source_list:
    query = ref_channel+source
    prefix_list = glob.glob("MYRiAD_V2/audio/*/*/*/"+query)
    prefix_list = [match[:-len(query)] for match in prefix_list]
    for prefix in prefix_list:
        x = [];
        try:
            for channel in channel_list:
                filename = prefix + channel + source
                x.append(torchaudio.load(filename)[0][0])
            x = torch.stack(x)
            x = 352.0*x
            x = x.to(torch.bfloat16)
            audio.append(x)
        except:
            pass

In [ ]:
audio = {"audio": audio};
audio_dataset = Dataset.from_dict(audio)

In [ ]:
audio_dataset.cast_column("audio", Audio())